# 1- import libraries

In [1]:
import numpy as np 
from collections import Counter 
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score

# 2- Decision Tree Class

In [2]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None
    
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
    
    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_labels = len(set(y))

        # Stopping criteria
        if depth >= self.max_depth or num_samples < self.min_samples_split or num_labels == 1:
            leaf_value = self._most_common_label(y)
            return leaf_value

        # Select the best split
        feat_idx, threshold = self._best_split(X, y, num_features)
        if feat_idx is None:
            return self._most_common_label(y)
        
        # Split the data
        left_idxs, right_idxs = self._split(X[:, feat_idx], threshold)
        left_tree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_tree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        
        return {"feature_index": feat_idx, "threshold": threshold, "left": left_tree, "right": right_tree}
    
    def _best_split(self, X, y, num_features):
        best_gain = -1
        split_idx, split_threshold = None, None
        
        for feat_idx in range(num_features):
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(X_column, y, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold
        
        return split_idx, split_threshold

    def _information_gain(self, X_column, y, threshold):
        # Parent loss (gini)
        parent_loss = self._gini(y)
        
        # Create children
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # Weighted average of the loss for the children
        num_left, num_right = len(left_idxs), len(right_idxs)
        num_total = len(y)
        left_loss = self._gini(y[left_idxs])
        right_loss = self._gini(y[right_idxs])
        child_loss = (num_left / num_total) * left_loss + (num_right / num_total) * right_loss
        
        # Information gain is difference in loss
        ig = parent_loss - child_loss
        return ig

    def _split(self, X_column, split_threshold):
        left_idxs = np.where(X_column <= split_threshold)[0]
        right_idxs = np.where(X_column > split_threshold)[0]
        return left_idxs, right_idxs
    
    def _gini(self, y):
        labels, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        gini = 1 - np.sum(probs ** 2)
        return gini
    
    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, tree):
        if not isinstance(tree, dict):
            return tree
        
        feature_value = x[tree["feature_index"]]
        if feature_value <= tree["threshold"]:
            return self._traverse_tree(x, tree["left"])
        else:
            return self._traverse_tree(x, tree["right"])


# 3- Random Forest Classifier 

In [3]:
class RandomForest:
    def __init__(self, n_trees=10, max_depth=None, min_samples_split=2, max_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.trees = []
    
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            
            # Bootstrap sampling
            X_sample, y_sample = self._bootstrap_sample(X, y)
            
            # Fit the decision tree
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
    
    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]
    
    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.array([self._most_common_label(tree_pred) for tree_pred in tree_preds.T])
    
    def _most_common_label(self, y):
        return Counter(y).most_common(1)[0][0]

# 4- implementation on Iris dataset

In [4]:
if __name__ == "__main__":
    data = load_iris()
    X, y = data.data, data.target

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train Random Forest
    forest = RandomForest(n_trees=10, max_depth=10)
    forest.fit(X_train, y_train)

    # Make predictions
    y_pred = forest.predict(X_test)

    # Evaluate model performance
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

Accuracy: 1.0000
